In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# Get rid of warnings (!)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
from IPython.display import clear_output

# Import all the deep learning stuff
# import tensorflow as tf
# import keras
# from tensorflow.keras import layers
# from sklearn.model_selection import train_test_split

# Import all the math and analysis stuff
import numpy as np
import matplotlib.pyplot as plt
import math

# Import models
from training.LSTM import LSTM
from training.ConvLSTM import ConvLSTM

In [3]:
# Import script for loading gestures
from datatools.data_process import load_preprocessed_data_split_on_candidate, load_candidate_map, divide_cand_split

In [4]:
import evaluation.convertion as evaluate
from evaluation.utils import get_timestr

# Model types
from training.ConvLSTM import ConvLSTM_horizontal
from training.ConvLSTM import ConvLSTM


In [ ]:
# Evaluate size of TF Lite model.
def size_evaluation_proceidure(training_data, model_builder, model_name, epochs):
    # Get data from tuple data
    X_train, X_test, y_train, y_test = training_data 

    model = model_builder()

    # Do training
    history = model.fit(
        X_train, 
        y_train, 
        batch_size=32,
        validation_split=0.1,
        shuffle=True, 
        epochs=epochs,
        verbose = 1
    )

    # Generate result string
    test_loss, test_acc = model.evaluate(np.array(X_test), np.array(y_test), verbose=2)
    res_string = f"Test loss: {test_loss}, Test acc: {test_acc}"

    # Save model to disk.
    model.save(f"./output/models/tf/{model_name}_{get_timestr()}.h5")
    clear_output()
    
    # Convertion and size
    file_name = evaluate.convert_and_quantize(model, X_train, model_name = model_name)
    clear_output()

    with open(f"{file_name}.txt", 'a') as file:
        file.write(res_string)

In [ ]:
# Evaluate stats versus validation stats

# Size stuff

## Horizontal kernels (3, 1)

In [ ]:
EPOCHS = 200
LEARNING_RATE = 0.00005

models_to_eval_horizontal = [
    ("ConvLSTM_one_conv_layers_horizontal_kernel_lstm_32units_framesize_5", lambda: ConvLSTM_horizontal.get_model_one_layer_framesize_5(units=32, lr=LEARNING_RATE)),
    ("ConvLSTM_one_conv_layers_horizontal_kernel_lstm_64units_framesize_5", lambda: ConvLSTM_horizontal.get_model_one_layer_framesize_5(units=64, lr=LEARNING_RATE)),
    ("ConvLSTM_one_conv_layers_horizontal_kernel_lstm_128units_framesize_5", lambda:ConvLSTM_horizontal.get_model_one_layer_framesize_5(units=128, lr=LEARNING_RATE)),
    ("ConvLSTM_two_conv_layers_horizontal_kernel_lstm_32units_framesize_5", lambda: ConvLSTM_horizontal.get_model_two_layers_framesize_5(units=32, lr=LEARNING_RATE)),
    ("ConvLSTM_two_conv_layers_horizontal_kernel_lstm_64units_framesize_5", lambda: ConvLSTM_horizontal.get_model_two_layers_framesize_5(units=64, lr=LEARNING_RATE)),
    ("ConvLSTM_two_conv_layers_horizontal_kernel_lstm_128units_framesize_5", lambda: ConvLSTM_horizontal.get_model_two_layers_framesize_5(units=128, lr=LEARNING_RATE)),
    ("expr_2_ConvLSTM_complex_horizontal_kernel_lstm_32units_framesize_5", lambda: ConvLSTM_horizontal.complex_model(units=32, lr=LEARNING_RATE)),
    ("expr_2_ConvLSTM_complex_horizontal_kernel_lstm_64units_framesize_5", lambda: ConvLSTM_horizontal.complex_model(units=64, lr=LEARNING_RATE)),
    ("expr_2_ConvLSTM_complex_horizontal_kernel_lstm_128units_framesize_5", lambda: ConvLSTM_horizontal.complex_model(units=128, lr=LEARNING_RATE)),
]

In [ ]:
from training.ConvLSTM import ConvLSTM

# Load data
X_train, X_test, y_train, y_test, enc = load_preprocessed_data_split_on_candidate(
    log=True,
    divide_into_frames=True,
    normalize_per_channel=True
)
training_data = (X_train, X_test, y_train, y_test)

# Do evalutaion proceidure
for model_name, model_builder in models_to_eval_horizontal:
    print(f"Evaluation of model: {model_name}")
    size_evaluation_proceidure(training_data, model_builder=model_builder, model_name = model_name, epochs = EPOCHS)


## Square kernels

In [ ]:
EPOCHS = 200
LEARNING_RATE = 0.00005

models_to_eval_square = [
    ("ConvLSTM_one_conv_layer_lstm_32units_framesize_5", lambda: ConvLSTM.get_model_one_conv_framesize_5(units=32, lr=LEARNING_RATE)),
    ("ConvLSTM_one_conv_layer_lstm_64units_framesize_5", lambda: ConvLSTM.get_model_one_conv_framesize_5(units=64, lr=LEARNING_RATE)),
    ("ConvLSTM_one_conv_layer_lstm_128units_framesize_5", lambda: ConvLSTM.get_model_one_conv_framesize_5(units=128, lr=LEARNING_RATE)),
    ("ConvLSTM_two_conv_layers_lstm_32units_framesize_5", lambda: ConvLSTM.get_model_two_conv_framesize_5(units=32, lr=LEARNING_RATE)),
    ("ConvLSTM_two_conv_layers_lstm_64units_framesize_5", lambda: ConvLSTM.get_model_two_conv_framesize_5(units=64, lr=LEARNING_RATE)),
    ("ConvLSTM_two_conv_layers_lstm_128units_framesize_5", lambda: ConvLSTM.get_model_two_conv_framesize_5(units=128, lr=LEARNING_RATE))
]
 

In [ ]:
# Load data
X_train, X_test, y_train, y_test, enc = load_preprocessed_data_split_on_candidate(
    log=True,
    divide_into_frames=True,
    normalize_per_channel=True
)
training_data = (X_train, X_test, y_train, y_test)

# Do evalutaion proceidure
for model_name, model_builder in models_to_eval_square:
    print(f"Evaluation of model: {model_name}")
    size_evaluation_proceidure(training_data, model_builder=model_builder, model_name = model_name, epochs = EPOCHS)


# Overfitting test

In [30]:
# Plot overfitting stuffs
def overfit_evaluation_proceidure(training_data, model_builder, model_name, epochs):
    # Get data from tuple data
    X_train, X_test, y_train, y_test = training_data 

    model = model_builder()

    # Do training
    history = model.fit(
        X_train, 
        y_train, 
        batch_size=32,
        validation_split=0.1,
        shuffle=True, 
        epochs=epochs,
        verbose = 1
    )

    # Generate result string
    test_loss, test_acc = model.evaluate(np.array(X_test), np.array(y_test), verbose=2)
    res_string = f"Test loss: {test_loss}, Test acc: {test_acc}"


    # Make and save plots
    Metrics.plot_history(history, model_name, res_string)

In [40]:
EPOCHS = 200 
UNITS = 128

models_to_eval_square = [
    (f"ConvLSTM_two_conv_layers_lstm_{UNITS}units_framesize_5_lr_0.000001", lambda: ConvLSTM.get_model_two_conv_framesize_5(units=UNITS, lr=0.000001)),
    (f"ConvLSTM_two_conv_layers_lstm_{UNITS}units_framesize_5_lr_0.000005", lambda: ConvLSTM.get_model_two_conv_framesize_5(units=UNITS, lr=0.000005)),
    (f"ConvLSTM_two_conv_layers_lstm_{UNITS}units_framesize_5_lr_0.00001", lambda: ConvLSTM.get_model_two_conv_framesize_5(units=UNITS, lr=0.00001)),
    (f"ConvLSTM_two_conv_layers_lstm_{UNITS}units_framesize_5_lr_0.00005", lambda: ConvLSTM.get_model_two_conv_framesize_5(units=UNITS, lr=0.00005)),
    (f"ConvLSTM_two_conv_layers_lstm_{UNITS}units_framesize_5_lr_0.0001", lambda: ConvLSTM.get_model_two_conv_framesize_5(units=UNITS, lr=0.0001)),
    (f"ConvLSTM_two_conv_layers_lstm_{UNITS}units_framesize_5_lr_0.0005", lambda: ConvLSTM.get_model_two_conv_framesize_5(units=UNITS, lr=0.0005))
]

In [41]:
from evaluation.metrics import Metrics

In [42]:
# Load data
X_train, X_test, y_train, y_test, enc = load_preprocessed_data_split_on_candidate(
    log=True,
    divide_into_frames=True,
    normalize_per_channel=True
)
training_data = (X_train, X_test, y_train, y_test)

# Do evalutaion proceidure
for model_name, model_builder in models_to_eval_square:
    print(f"Evaluation of model: {model_name}")
    overfit_evaluation_proceidure(training_data, model_builder=model_builder, model_name = f"{model_name}_400EPOCHS", epochs = 400)


Loading preprocessed data and splitting with the following parameters: hand: Hand.right, test_size: 0.1, divide into frames: True, shuffle: True
size of training set: 1650
size of test set: 220
Evaluation of model: ConvLSTM_two_conv_layers_lstm_128units_framesize_5_lr_0.000001
Epoch 1/400
47/47 [==============================] - 4s 34ms/step - loss: 2.3113 - acc: 0.1010 - val_loss: 2.2946 - val_acc: 0.1333
Epoch 2/400
47/47 [==============================] - 1s 20ms/step - loss: 2.3076 - acc: 0.1131 - val_loss: 2.2948 - val_acc: 0.1333
Epoch 3/400
47/47 [==============================] - 1s 19ms/step - loss: 2.3097 - acc: 0.0923 - val_loss: 2.2950 - val_acc: 0.1333
Epoch 4/400
47/47 [==============================] - 1s 19ms/step - loss: 2.3080 - acc: 0.0882 - val_loss: 2.2952 - val_acc: 0.1333
Epoch 5/400
47/47 [==============================] - 1s 18ms/step - loss: 2.3118 - acc: 0.1051 - val_loss: 2.2954 - val_acc: 0.1333
Epoch 6/400
47/47 [==============================] - 1s 20ms/s

In [43]:
# Do evalutaion proceidure
for model_name, model_builder in models_to_eval_square:
    print(f"Evaluation of model: {model_name}")
    overfit_evaluation_proceidure(training_data, model_builder=model_builder, model_name = f"{model_name}_200EPOCHS", epochs = 200)

Evaluation of model: ConvLSTM_two_conv_layers_lstm_128units_framesize_5_lr_0.000001
Epoch 1/200
47/47 [==============================] - 5s 35ms/step - loss: 2.3087 - acc: 0.1057 - val_loss: 2.3080 - val_acc: 0.0667
Epoch 2/200
47/47 [==============================] - 1s 25ms/step - loss: 2.3083 - acc: 0.1111 - val_loss: 2.3079 - val_acc: 0.0667
Epoch 3/200
47/47 [==============================] - 1s 27ms/step - loss: 2.3091 - acc: 0.0956 - val_loss: 2.3079 - val_acc: 0.0667
Epoch 4/200
47/47 [==============================] - 1s 28ms/step - loss: 2.3104 - acc: 0.1044 - val_loss: 2.3079 - val_acc: 0.0667
Epoch 5/200
47/47 [==============================] - 1s 24ms/step - loss: 2.3048 - acc: 0.1044 - val_loss: 2.3079 - val_acc: 0.0667
Epoch 6/200
47/47 [==============================] - 1s 25ms/step - loss: 2.3082 - acc: 0.0963 - val_loss: 2.3079 - val_acc: 0.0667
Epoch 7/200
47/47 [==============================] - 1s 28ms/step - loss: 2.3101 - acc: 0.0875 - val_loss: 2.3080 - val_acc:

# Square kernels


# Rest

In [ ]:
import tensorflow as tf
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# Save model to disk.
from evaluation.utils import get_timestr
model.save(f"./output/models/tf/{model_name}_{get_timestr()}.h5")
clear_output()

In [ ]:

model_name = "ConvLSTM_with_pooling_small_batches"

In [ ]:
import tensorflow as tf
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
test_loss, test_acc = model.evaluate(np.array(X_test), np.array(y_test), verbose=2)
print(f"Test loss: {test_loss}, Test acc: {test_acc}")